In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
training.head(1)

[Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658}))]

In [9]:
training.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [14]:
model = lr.fit(training)

In [15]:
model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [16]:
model.intercept

0.14228558260358093

In [17]:
training_summary = model.summary

In [18]:
training_summary.r2

0.027839179518600154

In [19]:
training_summary.rootMeanSquaredError

10.16309157133015

In [20]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [21]:
split_object = all_data.randomSplit([0.7, 0.3])

In [22]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [23]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [24]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 369|
|   mean|-0.17150943027058618|
| stddev|  10.197507983047531|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [26]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                132|
|   mean| 1.4544566643146695|
| stddev|  10.59465521194417|
|    min|-22.837460416919342|
|    max| 27.111027963108548|
+-------+-------------------+



In [27]:
new_model = lr.fit(train_data)

In [28]:
test_results = new_model.evaluate(test_data)

In [30]:
test_results.r2

-0.0380811919342412

In [31]:
test_results.residuals.show()

/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
|-18.566383584058403|
|-19.467961824745856|
|-19.129694471466948|
|-18.988218118824232|
|-18.570748703433818|
|-16.228867581437495|
| -16.04320139754446|
| -17.41549606652202|
| -15.28173361560563|
|-15.341607633641036|
|-14.936393932388539|
|-14.315139536379043|
|-12.198393289669877|
| -17.39666937115383|
| -10.30724481848718|
| -7.493915894650255|
|-11.760060858120877|
|-12.543365094528617|
| -7.215047213011688|
| -8.980490047210525|
+-------------------+
only showing top 20 rows



In [32]:
unlabelled_data = test_data.select('features')

In [34]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [35]:
predictions = new_model.transform(unlabelled_data)

In [38]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -4.271076832860937|
|(10,[0,1,2,3,4,5,...| -0.4050292133225501|
|(10,[0,1,2,3,4,5,...| -0.2726415587476056|
|(10,[0,1,2,3,4,5,...|  1.5595435478847253|
|(10,[0,1,2,3,4,5,...|  1.5053490775578022|
|(10,[0,1,2,3,4,5,...| -0.7976246827720535|
|(10,[0,1,2,3,4,5,...| 0.18119206997390075|
|(10,[0,1,2,3,4,5,...|    1.63481103389872|
|(10,[0,1,2,3,4,5,...|-0.45035465663361524|
|(10,[0,1,2,3,4,5,...| 0.28512465909860235|
|(10,[0,1,2,3,4,5,...|  0.6074154233130955|
|(10,[0,1,2,3,4,5,...|  0.3390086052263391|
|(10,[0,1,2,3,4,5,...| -1.6686946054888918|
|(10,[0,1,2,3,4,5,...|  3.6242278094509595|
|(10,[0,1,2,3,4,5,...| -2.1720353929643177|
|(10,[0,1,2,3,4,5,...|  -4.917030508189906|
|(10,[0,1,2,3,4,5,...|  -0.334290420414382|
|(10,[0,1,2,3,4,5,...|  1.2149491577508345|
|(10,[0,1,2,3,4,5,...|   -3.82430059524214|
|(10,[0,1,2,3,4,5,...| -1.315175